#### **Explanation:**
* Demonstrates multi-agent cooperation
* Agent roles are clearly divided: thinking vs doing
* Foundation for more advanced agent orchestration

In [ ]:
import time
import random

In [ ]:
# ----------------------------------------
# MessageBus: Handles communication between agents
# ----------------------------------------
class MessageBus:
    def __init__(self):
        self.messages = []

    # Send message from one agent to another
    def send(self, sender, receiver, content):
        self.messages.append({"from": sender, "to": receiver, "content": content})

    # Each agent fetches its own messages
    def fetch(self, agent_name):
        inbox = [msg for msg in self.messages if msg["to"] == agent_name]
        self.messages = [msg for msg in self.messages if msg["to"] != agent_name]
        return inbox

In [ ]:
# ----------------------------------------
# Base Agent Class: All agents inherit from this
# ----------------------------------------
class Agent:
    def __init__(self, name, bus):
        self.name = name
        self.bus = bus

    # Standard logging method for all agents
    def log(self, message, prefix="INFO"):
        print(f"[{self.name}][{prefix}] {message}")

    # Each agent must define its own behavior
    def act(self):
        raise NotImplementedError

In [ ]:
# ----------------------------------------
# UserAgent: Starts the system with user input
# ----------------------------------------
class UserAgent(Agent):
    def act(self):
        # Give clear choices so user knows what to input
        print("Choose a task to delegate to the system:")
        print("1. Plan a team meeting")
        print("2. Create a project report")
        print("3. Enter custom task")

        # Get input from user
        choice = input("Enter your choice (1/2/3): ")

        # Map choice to actual task
        if choice == "1":
            task = "plan a team meeting"
        elif choice == "2":
            task = "create a project report"
        else:
            task = input("Enter your custom task:\n> ")

        # Log and send task to PlannerAgent
        self.log(f"Received user task: '{task}'", "INPUT")
        self.log("Sending task to PlannerAgent...", "ACTION")
        self.bus.send(self.name, "PlannerAgent", {"type": "task", "data": task})

In [ ]:
# ----------------------------------------
# PlannerAgent: Breaks high-level task into smaller steps
# ----------------------------------------
class PlannerAgent(Agent):
    def act(self):
        for msg in self.bus.fetch(self.name):
            task = msg["content"]["data"]
            self.log(f"Received task from {msg['from']}: '{task}'", "RECEIVED")

            # Create task plan depending on the input
            if "meeting" in task.lower():
                plan = ["Check team availability", "Find common slot", "Book meeting room", "Send invites"]
            elif "report" in task.lower():
                plan = ["Collect data", "Analyze metrics", "Write summary", "Share report"]
            else:
                plan = ["Manual breakdown required", "Request clarification from user"]

            # Log and send the plan to the SchedulerAgent
            self.log(f"Created plan: {plan}", "PLANNING")
            self.log("Sending plan to SchedulerAgent...", "ACTION")
            self.bus.send(self.name, "SchedulerAgent", {"type": "plan", "data": plan})

In [ ]:
# ----------------------------------------
# SchedulerAgent: Assigns time slots to each step in the plan
# ----------------------------------------
class SchedulerAgent(Agent):
    def act(self):
        for msg in self.bus.fetch(self.name):
            plan = msg["content"]["data"]
            self.log(f"Received plan from {msg['from']}: {plan}", "RECEIVED")

            scheduled = []
            self.log("Scheduling tasks with random time slots...", "SCHEDULING")

            # Randomly assign a time slot to each step
            for step in plan:
                slot = f"{random.randint(9, 17)}:00"
                scheduled_step = f"{step} @ {slot}"
                self.log(f" : {scheduled_step}", "SCHEDULED")
                scheduled.append(scheduled_step)

            # Send scheduled steps to ExecutorAgent
            self.log("Sending scheduled steps to ExecutorAgent...", "ACTION")
            self.bus.send(self.name, "ExecutorAgent", {"type": "schedule", "data": scheduled})

In [ ]:
# ----------------------------------------
# ExecutorAgent: Executes each step in order
# ----------------------------------------
class ExecutorAgent(Agent):
    def __init__(self, name, bus):
        super().__init__(name, bus)
        self.done = False  # Set to True when task is complete

    def act(self):
        for msg in self.bus.fetch(self.name):
            steps = msg["content"]["data"]
            self.log(f"Received steps to execute from {msg['from']}", "RECEIVED")
            self.log("Beginning execution of steps...", "EXECUTION")

            # Simulate each step with a small delay
            for step in steps:
                self.log(f"Executing → {step}", "STEP")
                time.sleep(0.3)

            self.log("Execution complete.", "DONE")
            self.done = True  # Stop system after this

In [ ]:
# ----------------------------------------
# Main system loop: Runs all agents in order until task completes
# ----------------------------------------
def run_multi_agent_system():
    # Create message bus
    bus = MessageBus()

    # Initialize all agents
    user = UserAgent("UserAgent", bus)
    planner = PlannerAgent("PlannerAgent", bus)
    scheduler = SchedulerAgent("SchedulerAgent", bus)
    executor = ExecutorAgent("ExecutorAgent", bus)

    # Group agents for easy looping
    agents = [planner, scheduler, executor]

    print("=== Multi-Agent System Started ===\n")

    # Step 1: Get input and trigger flow
    user.act()

    # Step 2: Run cycles until execution is complete
    max_cycles = 10  # Fail-safe to prevent infinite loop
    for cycle in range(max_cycles):
        print(f"\n--- Agent Cycle {cycle+1} ---")
        for agent in agents:
            agent.act()

        # If final execution is done, shut down
        if executor.done:
            print("\nTask completed. System shutting down.")
            break

        time.sleep(0.5)

In [ ]:
run_multi_agent_system()